# Load Layer outputs

In [8]:
nn = 'LNN'

import pickle
with open(nn+'_layer_outputs.dat','rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()

# Load recording and test data

In [9]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 0.25

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

# Run Multiple Regressions

In [10]:
from copy import copy, deepcopy
from RE_PartialRecData2 import RE_PartialRecData2
from sklearn import preprocessing, linear_model
import pickle


# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
#subnetSize =[1, 2,8, 64]
nSubnetSize = len(subnetSize)
# which layers?
iLayer=0
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for iLayer in range(len(layer_outs)-1):
    for ss in range(nSubnetSize):
        #r_coefs =np.zeros((nIterations, 10, 128))
        #nLayerNeurons = [subnetSize[ss], 0, 0, 0, 10]
        nLayerNeurons = subnetSize[ss]
        print(subnetSize[ss])
        for it in range(nIterations):
            # copy data - is this necessary?
            layer_outputs = deepcopy(layer_outs)
            # subsample
            X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], \
                                                          nLayerNeurons, nRecordings, nSamples[ss])
            # impute X_subsample with mean value, apply imputation to test set
            imp =preprocessing.Imputer(missing_values='NaN', strategy='mean')
            impf =imp.fit(X_subsample)
            X_new = impf.transform(X_subsample)
            (rem_cols, ) = np.where(np.sum(X_new, axis=0)==0)
            if rem_cols.shape[0]>0:
                print('Removing cols #', rem_cols)
            X_test_new= impf.transform(layer_outs_test[iLayer])
            # fit regression model
            #regr = linear_model.Lasso(0.01)
            regr =linear_model.LinearRegression()
            regr.fit(X_new, Y_subsample)
            #r_coefs[it,:, ]=regr.coef_
            # calculate rmse
            rmses[it,:,ss] = np.sqrt(np.mean((regr.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))

            print ('predicting, mean RMSEs=%f' %np.mean(rmses[it, :, ss]))

    fName = 'results/LR_RMSES_Layer'+str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    # save the rmse's
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()

1
predicting, mean RMSEs=0.729523
predicting, mean RMSEs=0.606140
predicting, mean RMSEs=0.590773
predicting, mean RMSEs=0.502762
predicting, mean RMSEs=0.538829
2
predicting, mean RMSEs=1.049960
predicting, mean RMSEs=1.233182
predicting, mean RMSEs=1.016964
predicting, mean RMSEs=0.955506
predicting, mean RMSEs=1.406355
4
predicting, mean RMSEs=1.628554
predicting, mean RMSEs=2.135267
predicting, mean RMSEs=2.237410
predicting, mean RMSEs=1.850417
predicting, mean RMSEs=2.227274
8
predicting, mean RMSEs=2.555671
predicting, mean RMSEs=2.197685
predicting, mean RMSEs=2.614826
predicting, mean RMSEs=2.398388
predicting, mean RMSEs=2.721739
16
predicting, mean RMSEs=2.377649
predicting, mean RMSEs=2.294038
predicting, mean RMSEs=2.338274
predicting, mean RMSEs=2.062144
predicting, mean RMSEs=1.894791
32
predicting, mean RMSEs=1.386307
predicting, mean RMSEs=1.319907
predicting, mean RMSEs=1.123252
predicting, mean RMSEs=1.130593
predicting, mean RMSEs=1.021092
64
predicting, mean RMSEs=